In [39]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, col, element_at, slice, size, regexp_extract, transform, when, explode, \
monotonically_increasing_id, map_from_arrays, lit, udf,collect_list, row_number, ceil, map_keys, expr, from_json
from pyspark.sql.types import ShortType, ArrayType, LongType, StringType
from pyspark.sql import types as T
from pyspark.pandas import read_parquet
import pandas as pd
from os import listdir
from os.path import isfile, join

import os
import sys


years_Columns = list(range(1800,2000)) 
directory = 'C:/Users/bincl/BA-Thesis/Dataset/2gram/default/'
directory_par = 'C:/Users/bincl/BA-Thesis/Dataset/2gram/parquet/'

#pathlist = [directory + f for f in listdir(directory) if isfile(join(directory, f))]

spark2 = SparkSession.builder.appName('3gramSQL').getOrCreate()

df = spark2.read.option("header","true").option("recursiveFileLookup","true").parquet(directory_par)

In [ ]:
path = 'C:/Users/bincl/BA-Thesis/Dataset/3gram/default/3_20000_nopos_sample/3_20000_nopos_sample.gz'
directory_par = 'C:/Users/bincl/BA-Thesis/Dataset/2gram/parquet/'

spark = SparkSession.builder.appName('3gramSQL').getOrCreate()

raw_input_df = spark \
                .read.csv(path, sep='\n',quote="").withColumnRenamed('_c0', 'Input')

split_df = raw_input_df \
                .select(split('Input', '\t').alias('SplitInput')) \
                .select(element_at('SplitInput', 1).alias('Tokens'),
                        slice('SplitInput', 2,
size('SplitInput')).alias('Data')) \
                .select('Tokens', 'Data') \

df_3gram = split_df.select('Tokens', transform('Data', lambda d:
split(d, ',')).alias('Data')) \
                .select('Tokens', transform('Data', lambda x:
x[0]).alias('Year'),
                        transform('Data', lambda x:
x[1]).cast(ArrayType(LongType())).alias('Occurrences')) \
                .select('Tokens', map_from_arrays('Year',
'Occurrences').alias('Data')) \
                .select(['Tokens', 'Data'])          

In [40]:
#df = spark2.read.option("recursiveFileLookup","true").parquet("/path/to/root/")
#df.tail(3)

In [41]:

def get_pd_df_from_string(suchstring, df):
    x = df.select(['Tokens', 'Data']).where(df.Tokens == suchstring)
    x.printSchema()
    matched = x.first()['Data']
    matched_pandas_df = pd.Series(matched)
    matched_pandas_df.sort_index(inplace = True)

    matched_pandas_df.index.astype('int')
    numbers = list(range(1500,2020))
    numbers = map(str, numbers)
    matched_pandas_df = matched_pandas_df.reindex(numbers, fill_value= 0)
    return matched_pandas_df

In [42]:
# 1.2s mit parquet 15,5 ohne
matched_pandas_df = get_pd_df_from_string("Geschlecht '",df)
print(matched_pandas_df)

root
 |-- Tokens: string (nullable = true)
 |-- Data: map (nullable = true)
 |    |-- key: string
 |    |-- value: long (valueContainsNull = true)

1500      0
1501      0
1502      0
1503      0
1504      0
       ... 
2015    623
2016    423
2017    500
2018    409
2019    505
Length: 520, dtype: int64


In [43]:
#matched_pandas_df = get_pd_df_from_string('Titel beurteilt',df2)
#print(matched_pandas_df)

In [44]:
# if id set as default parameter then a list is given if not a spark dataframe is given as parameter df
def get_pd_df(df, id = -1):
    if id == -1 :
        matched = data['Data'] 
    else: 
        data = df.select(['Id', 'Data']).filter(df["Id"] == id).limit(1)
        matched = data.first()['Data']
    
    matched_pandas_df = pd.Series(matched)
    matched_pandas_df.sort_index(inplace = True)

    matched_pandas_df.index.astype('int')
    numbers = list(range(1500,2020))
    numbers = map(str, numbers)
    matched_pandas_df = matched_pandas_df.reindex(numbers, fill_value= 0)
    return matched_pandas_df

def get_pd_df_from_string(suchstring, df):
    x = df.select(['Id', 'Tokens'])
    x = x.where(x.Tokens == suchstring)
    id = x.first()['Id']
    print(id)
    return get_pd_df(df,id)      

In [45]:
print(get_pd_df(df_2gram, matched_pandas_df))

NameError: name 'df_2gram' is not defined

In [ ]:
rows = df_2gram.select(['Id']).count()
print(f"DataFrame Rows count : {rows}")
rows_2gram = 38123440
partition_id = rows_2gram/ 8

DataFrame Rows count : 38123440


In [ ]:
rows_2gram = 38123440
partition_id = 4765430
print(partition_id.)

4765430.0


In [ ]:
desired_row_number = 154619839732
desired_row = final_df.select(['Id', 'Data']).rdd.zipWithIndex().filter(lambda x: x[1] == desired_row_number).map(lambda x: x[0]).show()
 
print(desired_row)

In [ ]:
matched_pandas_df = get_pd_df_from_string('" Bis', df_2gram)
matched_pandas_df.plot()

154619839732
